# Introduction Sequential Modeling with Neural Networks



In [ ]:
import numpy as np
from numba import njit

import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-muted")

from tqdm import tqdm

In [ ]:
np.random.seed(72)

___

## Autoregressive Models of Order $p$ (AR($p$))

When modeling time series, we often encounter processes where the process at time $t-1$ might be useful in predicting the process at time $t$. A simple model that reflects such predictive power from past observations is the **autoregressive** model.

Let's delve into the structure of a simple autoregressive model of order $p$, or AR($p$) model. This model is described by the following equation:

$$
y_t = \phi_0 + \phi_1 y_{t-1} + \phi_2 y_{t-2} + \ldots + \phi_p y_{t-p} + \epsilon_t.
$$

In this model, $\epsilon_t$ represents a white noise process with mean zero and variance $\sigma_{\epsilon}^2$.

In [ ]:
# Generates n steps of AR process with coefficients (phi) (phi_0 = 0 by default)
@njit
def ar_process(phi: np.ndarray, sigma: float=1., n: int=300) -> np.ndarray:
    y = np.random.randn(n + len(phi)) * sigma
    for i in range(len(phi), n + len(phi)):
        y[i] = phi @ y[i - len(phi):i] + y[i]
    return y[len(phi):].reshape(-1, 1)

In [ ]:
# Plot four realizations of the AR process
fig, axs = plt.subplots(2, 2, figsize=(10, 3), sharex=True)
phis = map(np.array, [[0.6, -.2], [0.1, -.8], [0.3, .65], [0.3, 0.3]])

for i, ax in enumerate(axs.flatten()):
    phi = next(phis)
    ax.plot(ar_process(phi=phi, n=100))
    ax.set_title(r"$\phi_1 = {}, \phi_2 = {}$".format(*phi))
    # axs.set_ylim(-3, 3)
    ax.grid(alpha=0.3)

fig.suptitle("AR(2) Processes")
fig.tight_layout()

To keep things simple, let us first consider the AR(1) model, or autoregressive model of order 1:

$$
y_t = \phi_0 + \phi_1 y_{t-1} + \epsilon_t.
$$

This simple AR(1) model lets us discuss some interesting properties of AR models without going to deep into the details. In particular, conditional on the past observation ($y_{t-1}$), we can model the conditional expectation and variance of $y_t$ as

$$
\begin{align*}
\mathbb{E}[y_t | y_{t-1}] &= \mathbb{E}[\phi_0 + \phi_1 y_{t-1} + \epsilon_t] \\
&= \phi_0 + \phi_1 y_{t-1} \\
\mathrm{Var}[y_t | y_{t-1}] &= \sigma_{\epsilon}^2.
\end{align*}
$$

Hence, given the current realization $y_{t-1}$, the future realization is centered around $\phi_0 + \phi_1 y_{t-1}$ with a standard deviation of $\sigma_{\epsilon}$.

The AR process has many similarities with linear regression. In fact, we could simply estimate the $\phi$ coefficients using the least-squares method, e.g., for an AR process where we observe $T$ realizations, $y_1, y_2, \dots y_t$

$$\hat{\phi} = (X^\top X)^{-1} X^\top Y,$$

where $X = (y_{T-1}, y_{T-2}, \dots, y_1)^\top$ and $Y = (y_{T}, y_{T-1}, \dots, y_2)$.

In [ ]:
phi = np.array([0.3]) # Choose a non-explosive phi (|phi| < 1)

reps = 100 # Number of times we estimate the underlying phi

phi_hats = []
for i in range(reps):
    # Generate an AR sequence
    Ys = ar_process(phi)
    X, Y = Ys[:-1], Ys[1:]
    
    # Estimate parameter by least squares
    phi_hats.append(np.linalg.inv(X.T @ X) @ X.T @ Y)

print(f"True phi: {phi[0]:.4f}")
print(f"Estimate: {np.mean(phi_hats):.4f} ± {np.std(phi_hats) / np.sqrt(reps):.4f}")

Although the AR($p$) model generates data in a sequential manner, it can be effectively learned using static models, i.e., a simple linear regression or a feedforward neural network. The critical approach here is to utilize a sufficient number of lagged values of the target variable as features. This transformation enables these models to capture the dependencies and patterns inherent in the sequential data.

While the AR(p) model is a sequential data-generating process, it can easily be learned using a feedforward neural network (or a linear regression model for that matter). The key is to transform use sufficient lagged values of the target variable as features.

In considering an autoregressive model of the first order (AR(1)), represented by the equation $y_t = \phi_1 y_{t-1} + \epsilon_t$, it's apparent that the most accurate prediction of $y_t$ can be expressed as $\mathbb{E}[y_t | y_{t-1}] = \phi_1 y_{t-1}$. To solidify our understanding, let's explore whether a basic feedforward neural network can capture this relationship.

To proceed, we'll set a constant value for $\phi_1$ and generate data samples following the AR(1) model structure. Our goal is to employ a feedforward neural network, consisting of a single hidden layer, to effectively learn and model the relationship between $y_{t-1}$ (the input) and $y_t$ (the output)

___

## Feedforward Neural Networks (FFNNs)

Feedforward neural networks (FFNNs) are the simplest type of artificial neural network architecture. In an FFNN, the information moves in only one direction—forward—from the input nodes, through the hidden layers (if any), and finally to the output nodes. There are no cycles or loops in the network. This simplicity makes FFNNs a great starting point for understanding neural network architectures.

#### Structure of FFNNs
1. **Input Layer:** This layer consists of neurons that receive inputs and pass them on to the next layer. Each neuron in this layer represents one feature of the input data.
2. **Hidden Layers:** FFNNs can have one or multiple hidden layers. These layers are composed of neurons that apply weights to the inputs and pass them through a nonlinear activation function to produce outputs. The depth (number of layers) and width (number of neurons per layer) of an FFNN can vary, impacting its ability to learn complex patterns.
3. **Output Layer:** The final layer produces the output of the network. The structure of this layer depends on the specific task (e.g., regression, classification).

#### Activation Functions
Activation functions in FFNNs introduce non-linear properties to the model, enabling them to learn more complex patterns. Common activation functions include [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) (Rectified Linear Unit), [Sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function), [Tanh](https://en.wikipedia.org/wiki/Hyperbolic_functions), and [many others](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)

#### Sequence Modelling with FFNNs
Although FFNNs are not inherently designed for sequence modeling, they can still be effective in certain scenarios:
1. **Fixed-length Sequences**: When the sequence length is fixed and known, FFNNs can treat each element in the sequence as a separate feature. This approach works well for short sequences but can become impractical for longer ones due to the explosion in the number of input features.

2. **Feature Engineering**: Clever feature engineering can enable FFNNs to model sequences. For example, summarizing sequences with statistical measures (mean, standard deviation, etc.) or using lagged values can provide FFNNs with a better view of the sequence dynamics.

#### Basics of PyTorch

In case you are not familiar with writing neural networks in PyTorch, take some time to process the three code blocks below (and the documentation beyond that).

1. In the first code block, we simply import some necessary packages, for more information, please refer to the [PyTorch documentation](https://pytorch.org/docs/stable/index.html).
2. In the second code block, we define our neural network, a loss function, and an optimizer which will update the weights of our network.
3. In the third code block, we train our model. For each epoch, the sequence of steps is as follows:
    1. Set the gradients to zero
    2. Have the network run a forward pass (input &rarr; output)
    3. Compute the loss on the output produced in (2)
    4. Compute the gradients through backpropagation
    5. Update the weights of the neural network using our optimizer 


The 5 training steps happening in the third code block are paramount, and they **must** happen in that order (note: resetting the gradients may also happen after step 5 instead of at the beginning). Furthermore, they will take you very far when training neural networks in PyTorch.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# We can define a simple feedforward neural network with a single hidden layer
# of 32 units and a ReLU activation function as follows:
ffnn = nn.Sequential(
    nn.Linear(1, 32),   # Input layer (1 -> 32)
    nn.ReLU(),          # Activation function (applied element-wise)
    nn.Linear(32, 1)    # Output layer (32 -> 1)
)

# Define the loss function (MSE) and optimizer (stochastic gradient descent)
loss_fn = nn.MSELoss()
optimizer = optim.SGD(ffnn.parameters(), lr=0.01)

# Best predictor E[y | y_{t-1}] = phi * y_{t-1}
def best_predictor(phi, y): 
    return phi * y


In [ ]:
n_epochs = 500

# Fix AR(1) process parameters, feel free to change them (beware of stationarity!)
phi = np.array([0.5])

# Generate 100 AR(1) processes for validation
y_val = np.hstack([ar_process(phi) for _ in range(100)])

for epoch in tqdm(range(n_epochs)):
    # Generate a new AR(1) process for each epoch
    y = ar_process(phi)
    y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
    
    # Reset the gradients
    optimizer.zero_grad()
    
    # Compute the forward pass
    y_pred = ffnn(y[:-1])
    
    # Compute the loss
    loss = loss_fn(y_pred, y[1:])
    
    # Compute the gradients
    loss.backward()
    
    # Update the parameters
    optimizer.step()

# Compute validation loss
loss = 0
for i in range(y_val.shape[1]):
    y = torch.tensor(y_val[:, i], dtype=torch.float32).reshape(-1, 1)
    y_pred = ffnn(y[:-1])
    loss += loss_fn(y_pred, y[1:]).item()
loss /= y_val.shape[1]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))

xs = np.linspace(-1, 1, 250)
ax.plot(xs, phi * xs, label=r"$\mathbb{E}[y_{t} \mid y_{t-1}]$")
ax.plot(xs, ffnn(torch.tensor(xs, dtype=torch.float32).reshape(-1, 1)).detach().numpy(), 
        label="FFNN($y_{t-1}$)")
ax.grid(alpha=0.3)
ax.set_xlabel(r"$y_{t-1}$")
ax.set_ylabel(r"$\hat{y}_{t}$")
ax.set_title("Best predictor vs. FFNN")
ax.legend()

We see from the above plot, that the FFNN is able to learn a function that is very close to the best predictor of the relationship between $y_{t-1}$ and $y_t$. This is not surprising since the best predictor is a linear function of $y_{t-1}$. This is also suggested by the mean squared error (MSE) on the validation set.

In [ ]:
print(f"MSE of best predictor: {np.mean((y_val[:-1] - phi * y_val[1:])**2):.5f}")
print(f"MSE of FFNN          : {loss:.5f}")



But what happens if we increase the number of lags? Let us try this for an AR(2) model. In such a case, the best predictor of $y_t$ is $\mathbb{E}[y_t | y_{t-1}, y_{t-2}] = \phi_1 y_{t-1} + \phi_2 y_{t-2}$. Thus, the neural network will have to learn a linear function of two variables. 

Let us create two networks, one which uses a single lag and another which uses two lags. We will then compare the performance of the two networks on the validation set.

In [ ]:
# Define the models, and optimizers
ffnn_1 = nn.Sequential(nn.Linear(1, 32), nn.ReLU(), nn.Linear(32, 1))
ffnn_2 = nn.Sequential(nn.Linear(2, 32), nn.ReLU(), nn.Linear(32, 1))

optimizer_1 = optim.SGD(ffnn_1.parameters(), lr=0.01)
optimizer_2 = optim.SGD(ffnn_2.parameters(), lr=0.01)

In [ ]:
n_epochs = 500

# Fix AR(2) process parameters, feel free to change them (beware of stationarity!)
phi = np.array([0.1, -.4])

# Generate 100 AR(1) processes for validation
y_val = np.hstack([ar_process(phi) for _ in range(100)])

for epoch in tqdm(range(n_epochs)):
    # Generate a new AR(1) process for each epoch
    y = ar_process(phi, n=250)
    y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
    
    # Reset the gradients
    optimizer_1.zero_grad()
    optimizer_2.zero_grad()
    
    # Compute the forward pass
    y_pred_1 = ffnn_1(y[:-2])
    y_pred_2 = ffnn_2(torch.cat([y[:-2], y[1:-1]], dim=1))
    
    # Compute the loss
    loss_1 = loss_fn(y_pred_1, y[2:])
    loss_2 = loss_fn(y_pred_2, y[2:])
    
    # Compute the gradients
    loss_1.backward()
    loss_2.backward()
    
    # Update the parameters
    optimizer_1.step()
    optimizer_2.step()

# Compute validation loss
loss_1 = 0
loss_2 = 0
for i in range(y_val.shape[1]):
    y = torch.tensor(y_val[:, i], dtype=torch.float32).reshape(-1, 1)
    y_pred_1 = ffnn_1(y[:-2])
    y_pred_2 = ffnn_2(torch.cat([y[:-2], y[1:-1]], dim=1))
    loss_1 += loss_fn(y_pred_1, y[2:]).item()
    loss_2 += loss_fn(y_pred_2, y[2:]).item()
loss_1 /= y_val.shape[1]
loss_2 /= y_val.shape[1]
loss_best = np.mean((y_val[2:, :] - phi[1] * y_val[1:-1, :] - phi[0] * y_val[:-2, :])**2)

print(f"MSE of best predictor: {loss_best:.5f}")
print(f"MSE of FFNN (1-lag)  : {loss_1:.5f}")
print(f"MSE of FFNN (2-lag)  : {loss_2:.5f}")

While we notice that with enough lags, the feedforward neural network is able to rival the best predictor, we also notice that the neural network that uses a single lag as features is unable to learn the relationship as well as the neural network that uses two lags as features, which is expected since the autoregressive model is of order 2.

What implications does this illustrative example hold? It demonstrates that FFNNs are capable of learning from sequential data-generating processes. Yet, the choice of how many lags to use as features is crucial. There are several important considerations when using FFNNs for this purpose:

1. **Cost of Adding Lags:** Each additional lag used as a feature not only reduces the available sample size for training but also enlarges the network’s input size, consequently increasing the number of parameters that need to be learned.
2. **Handling Variable Sequence Length:** In many scenarios, we observe sequences of varying lengths, but FFNNs require a fixed input size. This necessitates choosing a uniform sequence length for input, which can involve padding or truncating sequences, potentially diminishing our network's performance. Ideally, a model that can adapt to varying sequence lengths would be preferable.
3. **Selecting the Number of Lags:**  In our toy example, using $p$ lags to learn an AR($p$) model was straightforward. However, in real-world scenarios, determining the optimal number of lags is often more complex and not as straightforward.
4. **FFNNs and Complex Sequences:** FFNNs may face challenges with certain sequences, especially where the relative positioning of elements is crucial. In a [blog post](https://www.jldc.ch/post/seq2one-flux/) I wrote, I discuss an example where an FFNN with two hidden layers struggles to learn the deterministic relationship between a 12-element input sequence and a single output variable, despite having access to the entire sequence. The post provides more detailed insights into this issue.

While FFNNs can be adapted for sequence modeling with some ingenuity, their limitations make them less suitable for complex sequence modeling tasks. These tasks are often better handled by architectures specifically designed for sequence data, such as recurrent neural networks (RNNs) and their extensions like long short-term memory networks (LSTMs) and gated recurrent units (GRUs), which we will discuss next. Understanding FFNNs, however, provides a fundamental basis for grasping these more advanced concepts in deep learning.

___

## A Twist on the AR($p$) Model

To showcase the limitations in sequence modeling, we don't even need to introduce a complex sequence. Let us stick with our simple AR($p$) model but introduce a twist. In the above example, we have implicitly assumed that we observe different realizations of AR($p$) process where all realizations have same value of $\phi_1, \ldots, \phi_p$. Naturally, in many real-world scenarios, we may observe different realizations of AR($p$) process where each realization has different values of $\phi_1, \ldots, \phi_p$. In such a case, we can no longer use a single feedforward neural network to learn the relationship between $y_{t-1}, \ldots, y_{t-p}$ and $y_t$.

In [ ]:
# Define the model, and optimizer
ffnn = nn.Sequential(nn.Linear(1, 32), nn.ReLU(), nn.Linear(32, 1))

optimizer = optim.SGD(ffnn.parameters(), lr=0.01)

In [ ]:
# Make a helper to generate batched data from the AR process
def batched_ar_process(phi_batch):
    # Batched size can be infered from phi
    y = np.hstack([ar_process(np.array([phi])) for phi in phi_batch]) 
    y = y[..., None] # Add a trailing dimension for input size (seq_len, batch_size, input_size)
    return y

In [ ]:
n_epochs = 500

# Generate 100 AR(1) processes for validation
phis_val = np.random.uniform(-1, 1, size=100)
y_val = np.hstack([ar_process(np.array([phi])) for phi in phis_val])

for epoch in tqdm(range(n_epochs)):
    # Generate a new AR(1) process for each epoch
    phi = np.random.uniform(-1, 1, size=1)
    y = ar_process(phi)
    y = torch.Tensor(y)
    
    # Reset the gradients
    optimizer.zero_grad()
    
    # Compute the forward pass
    y_pred = ffnn(y[:-1])
    
    # Compute the loss
    loss = loss_fn(y_pred, y[1:])
    
    # Compute the gradients
    loss.backward()
    
    # Update the parameters
    optimizer.step()

# Compute validation loss
loss = 0
for i in range(y_val.shape[1]):
    y = torch.tensor(y_val[:, i], dtype=torch.float32).reshape(-1, 1)
    y_pred = ffnn(y[:-1])
    loss += loss_fn(y_pred, y[1:]).item()
loss /= y_val.shape[1]
loss_best = np.mean((y_val[1:, :] - phis_val * y_val[:-1, :])**2)

print(f"MSE of best predictor: {loss_best:.5f}")
print(f"MSE of FFNN (1-lag)  : {loss:.5f}")

We notice the major difference in MSE between the best predictor and the FFNN for this particular scenario. This is because, in isolation, $y_{t-1}$ contains no information about $\phi_1.$ This is crucial. When practicing sequence modeling, we seldom have the luxury to assume that every realization of the sequence-generating process has the same parameters. In fact, in many real-world scenarios, we may not even know the parameters of the sequence-generating process. 

While we could use a separate FFNN for each realization of the sequence-generating process, this approach is not really scalable and forces us to keep a training set for each realization.
This is where recurrent neural networks (RNNs) come in.

___

## Recurrent Neural Networks (RNNs)

Recurrent Neural Networks (RNNs) are a class of artificial neural networks specifically designed to work with sequences of data, such as text, genomes, handwriting, or spoken words. Unlike FFNNs, RNNs have a unique feature: they have internal memory. This memory helps them remember previous inputs in the network's internal state, making them ideal for processing and predicting sequences.

#### Structure of RNNs
1. **Input Layer:** Similar to FFNNs, the input layer in RNNs consists of neurons that receive inputs. However, in RNNs, these inputs are processed sequentially.
2. **Hidden Layers (Recurrent Layers):** The core of an RNN is its hidden layers, which are also known as recurrent layers. In these layers, neurons not only receive input from the previous layer but also receive feedback from their own output. Each neuron in a recurrent layer processes one element of the input sequence at a time while maintaining an internal state from the previous elements of the sequence. This architecture enables RNNs to exhibit dynamic temporal behavior.
3. **Output Layer:** The output layer in an RNN can be structured to produce a single output at the end of the sequence or an output at each time step of the sequence, depending on the task at hand (e.g., sequence classification, sequence generation).

#### Sequence Modelling with RNNs
RNNs are inherently suited for sequence modeling due to their internal memory:
1. **Variable-length Sequences:** One of the key advantages of RNNs is their ability to handle sequences of variable lengths, unlike FFNNs. This is due to their recurrent nature, with the input sequence being handled one element at a time.

2. **Temporal Dynamics:** RNNs can capture the temporal dynamics of sequences, making them suitable for tasks like time series forecasting, language modeling, and speech recognition.

3. **Challenges:** While RNNs are powerful for sequence modeling, they are not without challenges. We will discuss these challenges in more detail in the next notebook.

In [ ]:
# Define the RNN class, we need to discard the hidden state at prediction time
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        # Define the layers
        self.rnn = nn.RNN(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)

    # Define the forward pass to remove the hidden state from the output
    def forward(self, x):
        # Apply the RNN
        x, _ = self.rnn(x) # (output, hidden state), discard the hidden state
        # Apply the linear layer
        x = self.linear(x)
        return x

In [ ]:
# Define the models, and optimizers
ffnn = nn.Sequential(nn.Linear(1, 32), nn.ReLU(), nn.Linear(32, 1))
rnn = SimpleRNN(1, 32, 1)

# RNN can be tricky to train, so we will use another optimizer and a different
# learning rate. For comparison, we will also train the FFNN with the same optimizer

opt_ffnn = optim.Adam(ffnn.parameters(), lr=0.001)
opt_rnn = optim.Adam(rnn.parameters(), lr=0.001)

In [ ]:
n_epochs = 500
batch_size = 64 # Learning is more stable with batched data

# Generate 100 AR(1) processes for validation
phis_val = np.random.uniform(-1, 1, size=100)
# phis_val = np.array([0.9 for _ in range(100)])
y_val = np.hstack([ar_process(np.array([phi])) for phi in phis_val])

for epoch in tqdm(range(n_epochs)):
    # Generate a batch of AR(1) process for each epoch
    phi = np.random.uniform(-1, 1, size=batch_size)
    # phi = np.array([0.9 for _ in range(batch_size)])
    y = batched_ar_process(phi)
    y = torch.Tensor(y)
    
    # Reset the gradients
    opt_ffnn.zero_grad()
    opt_rnn.zero_grad()
    
    # Compute the forward pass
    y_ffnn = ffnn(y[:-1])
    y_rnn = rnn(y[:-1])
    
    # Compute the loss (drop first component for warmup)
    loss_ffnn = loss_fn(y_ffnn, y[1:])
    loss_rnn = loss_fn(y_rnn, y[1:])
    
    # Compute the gradients
    loss_ffnn.backward()
    loss_rnn.backward()

    # Update the parameters
    opt_ffnn.step()
    opt_rnn.step()

# Compute validation losses
loss_ffnn = 0
loss_rnn = 0
rnn.eval()

y = torch.Tensor(y_val)[..., None]
y_ffnn = ffnn(y[:-1])
y_rnn = rnn(y[:-1])
loss_ffnn = loss_fn(y_ffnn, y[1:]).item()
loss_rnn = loss_fn(y_rnn, y[1:]).item()
loss_best = np.mean((y_val[1:] - phis_val * y_val[:-1])**2)

print(f"MSE of best predictor: {loss_best:.5f}")
print(f"MSE of FFNN (1-lag)  : {loss_ffnn:.5f}")
print(f"MSE of RNN           : {loss_rnn:.5f}")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))

phi = 0.7
y = ar_process(np.array([phi]), sigma=0.25)

# Plot true process and best predictor
ax.plot(y[1:], label="True", alpha=0.7)
ax.plot(y[:-1] * phi, label="Best", alpha=0.7)

# Plot neural network predictions
y = torch.Tensor(y).reshape(-1, 1, 1)
ax.plot(ffnn(y[:-1]).flatten().detach().numpy(), label="FFNN")
ax.plot(rnn(y[:-1]).flatten().detach().numpy(), label="RNN")

ax.grid(alpha=0.3)
fig.legend(ax.get_legend_handles_labels()[0], ax.get_legend_handles_labels()[1], 
           loc='upper center', ncol=4)

While the Recurrent Neural Network (RNN) may not match the performance of the best predictor in our scenario, it's important to consider the context of this comparison. The best predictor has an advantage as it is informed of the true value of $\phi_1$, simplifying its task significantly. In contrast, the RNN must deduce or learn the value of $\phi_1$ while it processes the sequence. Despite this challenge, we observe that the RNN demonstrates a superior ability to discern the relationship between $y_t$ and $y_{t-1}$ when compared to the FFNN.

This example highlights a situation where a hidden factor, in this case, the parameter $\phi_1$, influences each realization of the time series. Here, the RNN's capability to infer the latent factor as it processes data is a notable advantage over FFNNs. This attribute is particularly valuable in scenarios where learning underlying dynamics is crucial.

It's also worth noting that vanilla RNNs represent just one form within a broader family of RNN architectures. There are several advanced variants, such as Long Short-Term Memory networks (LSTMs) and Gated Recurrent Units (GRUs), which are engineered to overcome certain limitations of the basic RNN model, including issues like the vanishing gradient problem. These advanced RNN variants offer enhanced performance in handling complex sequences, and we will delve into these in the subsequent notebook

#### Pause and Ponder

Since we know how to formulate the best predictor for an AR($p$) process, is there also a way we could use this knowledge (and the knowledge we have about the data-generating process, i.e., $\phi$ is different for each realization). Can you come up with your own algorithm to predict the next step of any observed AR process in the above situation? This algorithm should not use a learner more advanced than the closed form OLS equation used above.

Hint: think of a sequence of least-squares estimate.